# load data

In [1]:
from glob import glob
import os
images=[]
labels=[]
for folder in os.listdir('../dataset/wrist_xray'):
    for pic in glob('../dataset/wrist_xray/{}/*.jpg'.format(folder)):
        images.append(pic)
        labels.append(folder)
        

In [2]:
import pandas as pd
data=pd.DataFrame(zip(images,labels),columns=['img','label'])
data.label=data.label.map({'Fracture':0,'Normal':1})
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,img,label
0,../dataset/wrist_xray/Fracture\27 Male (L View...,0
1,../dataset/wrist_xray/Fracture\60 F (L View).jpg,0
2,../dataset/wrist_xray/Fracture\59 F (L View).jpg,0
3,../dataset/wrist_xray/Normal\13.jpg,1
4,../dataset/wrist_xray/Normal\43.jpg,1


# top 5 lightest models in torchvision
1	squeezenet1_1	1235496  
2	shufflenet_v2_x0_5	1366792  
3	mnasnet0_5	2218512  
4	mobilenet_v2	3504872  
5	densenet121	7978856  `

In [3]:
import sys
sys.path.append("../src") 

In [4]:
from models.squeezenet import squeezenet1_1
from models.mymodels import mymodel4,mymodel3
from models.shufflenet import shufflenet_v2_x0_5
from models.mnastnet import mnasnet0_5
from models.densenet import densenet121
from models.mobilenet import mobilenet
from cross_vals import kfoldcv

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)/1e6

d:\tutorials\automl\venv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
print('MyNet parameters :' ,count_parameters(mymodel4()))
print('SqueezeNet parameters :', count_parameters(squeezenet1_1()))
print('ShuffleNet parameters :', count_parameters(shufflenet_v2_x0_5()))
print('MnastNet parameters :', count_parameters(mnasnet0_5()))
print('MobileNet parameters :', count_parameters(mobilenet()))
print('DenseNet parameters :', count_parameters(densenet121()))


MyNet parameters : 0.391408
SqueezeNet parameters : 0.787008
ShuffleNet parameters : 0.47256
MnastNet parameters : 1.101192
MobileNet parameters : 2.387264
DenseNet parameters : 7.078784


In [7]:
epoch=32
batchsize=8
lr=0.001

dim='3D' or '2D' for 3D images and 2D iamges
skip_tuning=False, no grid search applied for hyper-parameter tuning, 
aug=1-> pytorch augmentation
aug=0-> no augmentation
aug=2-> albumentation augmentation
aug=3-> rand augmentation

# MyNet

In [8]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel3(),epochs=epoch,batchsize=batchsize,data=data,\
                            lr=0.001,skip_tuning=False,aug=1,dim='2D',pca=False)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.896   0.840     0.860
Class 1       0.792   0.868     0.822
macro         0.844   0.852     0.840
weighted      0.862   0.846     0.846
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.916   0.860     0.884
Class 1       0.818   0.908     0.858
macro         0.870   0.882     0.874
weighted      0.886   0.874     0.874
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.920   0.888     0.900
Class 1       0.848   0.908     0.874
macro         0.884   0.896     0.890
weighted      0.900   0.890     0.890
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.920   0.880     0.896
Class 1       0.838   0.912     0.870
macro         0.880   0.894     0.884
weighted      0.896   0.886     0.886
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.77,0.87,0.92,0.87,0.95
1,0.87,0.95,0.95,0.95,0.97
2,0.90,0.87,0.90,0.87,0.90
3,0.82,0.84,0.84,0.87,0.89
4,0.87,0.84,0.84,0.87,0.89
mean,0.846,0.874,0.89,0.886,0.92


In [9]:
%%time
train_cv,clf_report=kfoldcv(model=mymodel3(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.636   0.718     0.666
Class 1       0.554   0.450     0.484
macro         0.592   0.582     0.572
weighted      0.618   0.596     0.592
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.768   0.728     0.736
Class 1       0.662   0.710     0.674
macro         0.714   0.720     0.706
weighted      0.738   0.710     0.710
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.794   0.742     0.750
Class 1       0.680   0.740     0.692
macro         0.734   0.740     0.720
weighted      0.764   0.728     0.726
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.668   0.688     0.670
Class 1       0.586   0.544     0.550
macro         0.626   0.620     0.610
weighted      0.650   0.628     0.626
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.54,0.67,0.62,0.59,0.77
1,0.69,0.82,0.90,0.77,0.85
2,0.64,0.64,0.67,0.51,0.69
3,0.58,0.76,0.74,0.66,0.79
4,0.53,0.66,0.71,0.61,0.76
mean,0.596,0.71,0.728,0.628,0.772


# SqueezeNet

In [10]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.672   0.742     0.700
Class 1       0.568   0.506     0.528
macro         0.618   0.626     0.614
weighted      0.636   0.644     0.632
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.648   0.654     0.646
Class 1       0.568   0.558     0.558
macro         0.610   0.606     0.604
weighted      0.624   0.610     0.612
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.658   0.662     0.658
Class 1       0.560   0.560     0.556
macro         0.610   0.610     0.608
weighted      0.630   0.620     0.620
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.630   0.706     0.662
Class 1       0.530   0.456     0.482
macro         0.582   0.582     0.572
weighted      0.602   0.596     0.592
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,1.00,0.97,0.97,0.95,1.00
1,0.67,0.51,0.46,0.44,0.59
2,0.44,0.46,0.54,0.49,0.54
3,0.58,0.66,0.63,0.55,0.66
4,0.53,0.45,0.50,0.55,0.66
mean,0.644,0.61,0.62,0.596,0.69


In [11]:
%%time
train_cv,clf_report=kfoldcv(model=squeezenet1_1(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.644   0.700     0.662
Class 1       0.538   0.484     0.498
macro         0.594   0.592     0.580
weighted      0.610   0.602     0.590
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.602   0.608     0.604
Class 1       0.492   0.484     0.486
macro         0.548   0.546     0.546
weighted      0.566   0.562     0.564
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.636   0.632     0.628
Class 1       0.500   0.508     0.496
macro         0.568   0.572     0.562
weighted      0.590   0.576     0.574
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.620   0.704     0.654
Class 1       0.492   0.408     0.440
macro         0.554   0.556     0.548
weighted      0.576   0.584     0.570
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.85,0.79,0.77,0.72,0.85
1,0.56,0.51,0.51,0.49,0.62
2,0.36,0.38,0.44,0.49,0.51
3,0.66,0.55,0.58,0.61,0.74
4,0.58,0.58,0.58,0.61,0.66
mean,0.602,0.562,0.576,0.584,0.676


# ShuffleNet

In [12]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.660   0.684     0.666
Class 1       0.526   0.510     0.510
macro         0.592   0.596     0.590
weighted      0.620   0.612     0.608
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.648   0.710     0.672
Class 1       0.534   0.470     0.492
macro         0.590   0.588     0.580
weighted      0.612   0.606     0.602
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.644   0.704     0.668
Class 1       0.526   0.466     0.490
macro         0.584   0.586     0.578
weighted      0.608   0.600     0.598
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.658   0.746     0.694
Class 1       0.546   0.452     0.482
macro         0.604   0.596     0.588
weighted      0.626   0.622     0.612
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.56,0.59,0.56,0.64,0.69
1,0.46,0.56,0.56,0.51,0.69
2,0.59,0.51,0.54,0.56,0.64
3,0.66,0.61,0.58,0.58,0.68
4,0.79,0.76,0.76,0.82,0.82
mean,0.612,0.606,0.6,0.622,0.704


In [13]:
%%time
train_cv,clf_report=kfoldcv(model=shufflenet_v2_x0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.582   0.626     0.598
Class 1       0.436   0.392     0.408
macro         0.508   0.508     0.500
weighted      0.536   0.524     0.524
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.642   0.744     0.682
Class 1       0.548   0.432     0.470
macro         0.594   0.588     0.578
weighted      0.616   0.608     0.596
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.622   0.694     0.644
Class 1       0.530   0.436     0.458
macro         0.574   0.566     0.550
weighted      0.600   0.582     0.572
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.584   0.642     0.588
Class 1       0.316   0.346     0.318
macro         0.450   0.492     0.450
weighted      0.468   0.492     0.462
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.59,0.59,0.51,0.59,0.69
1,0.49,0.62,0.59,0.54,0.64
2,0.51,0.56,0.62,0.33,0.64
3,0.50,0.61,0.66,0.61,0.71
4,0.53,0.66,0.53,0.39,0.74
mean,0.524,0.608,0.582,0.492,0.684


# Mnasnet

In [14]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.808   0.798     0.796
Class 1       0.742   0.746     0.736
macro         0.772   0.768     0.764
weighted      0.790   0.776     0.778
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.840   0.798     0.812
Class 1       0.756   0.796     0.764
macro         0.800   0.794     0.788
weighted      0.812   0.792     0.796
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.852    0.79     0.814
Class 1       0.756    0.81     0.772
macro         0.804    0.80     0.792
weighted      0.820    0.80     0.802
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.808   0.758     0.778
Class 1       0.730   0.774     0.740
macro         0.770   0.764     0.758
weighted      0.784   0.762     0.764
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.44,0.46,0.54,0.41,0.67
1,0.90,0.92,0.92,0.92,0.92
2,0.85,0.87,0.85,0.82,0.87
3,0.74,0.76,0.74,0.71,0.79
4,0.95,0.95,0.95,0.95,0.95
mean,0.776,0.792,0.8,0.762,0.84


In [15]:
%%time
train_cv,clf_report=kfoldcv(model=mnasnet0_5(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.614   0.668     0.618
Class 1       0.508   0.434     0.422
macro         0.564   0.552     0.524
weighted      0.590   0.544     0.530
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.578   0.638     0.600
Class 1       0.402   0.350     0.366
macro         0.492   0.496     0.484
weighted      0.514   0.512     0.506
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.620   0.704     0.654
Class 1       0.496   0.396     0.432
macro         0.556   0.548     0.542
weighted      0.576   0.574     0.566
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.616   0.662     0.626
Class 1       0.482   0.428     0.434
macro         0.550   0.542     0.530
weighted      0.574   0.560     0.548
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.59,0.46,0.46,0.67,0.64
1,0.62,0.62,0.74,0.54,0.72
2,0.46,0.38,0.38,0.41,0.62
3,0.55,0.55,0.61,0.55,0.68
4,0.50,0.55,0.68,0.63,0.66
mean,0.544,0.512,0.574,0.56,0.664


# MobileNet

In [16]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.884   0.860     0.870
Class 1       0.802   0.830     0.808
macro         0.842   0.846     0.838
weighted      0.856   0.846     0.846
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.934   0.908     0.918
Class 1       0.874   0.916     0.886
macro         0.902   0.914     0.904
weighted      0.912   0.908     0.908
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.928   0.924     0.924
Class 1       0.892   0.902     0.892
macro         0.908   0.914     0.908
weighted      0.918   0.912     0.912
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.896   0.940     0.914
Class 1       0.910   0.836     0.858
macro         0.902   0.888     0.886
weighted      0.908   0.894     0.892
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.54,0.85,0.85,0.79,0.87
1,0.92,0.95,0.95,0.95,0.97
2,0.95,0.95,0.95,0.92,0.97
3,0.95,0.95,0.97,0.97,0.97
4,0.87,0.84,0.84,0.84,0.87
mean,0.846,0.908,0.912,0.894,0.93


In [17]:
%%time
train_cv,clf_report=kfoldcv(model=mobilenet(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.648   0.696     0.668
Class 1       0.542   0.488     0.508
macro         0.596   0.590     0.588
weighted      0.616   0.612     0.608
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.662   0.870     0.748
Class 1       0.690   0.398     0.498
macro         0.678   0.636     0.624
weighted      0.684   0.672     0.650
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.636   0.778     0.692
Class 1       0.634   0.416     0.482
macro         0.634   0.594     0.590
weighted      0.644   0.628     0.612
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.648   0.636     0.626
Class 1       0.518   0.520     0.494
macro         0.584   0.580     0.560
weighted      0.610   0.574     0.572
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.51,0.67,0.72,0.51,0.72
1,0.67,0.74,0.67,0.69,0.79
2,0.64,0.56,0.54,0.56,0.74
3,0.63,0.71,0.71,0.50,0.74
4,0.61,0.68,0.50,0.61,0.68
mean,0.612,0.672,0.628,0.574,0.734


# DenseNet121

In [18]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=1)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

------------------ Decision Tree ----------------
          precision  recall  f1-score
Class 0       0.812   0.780     0.792
Class 1       0.718   0.770     0.734
macro         0.766   0.778     0.762
weighted      0.788   0.766     0.768
------------------ SVM ----------------
          precision  recall  f1-score
Class 0       0.842   0.816     0.824
Class 1       0.764   0.808     0.778
macro         0.804   0.810     0.802
weighted      0.820   0.808     0.808
------------------ logistic regression ----------------
          precision  recall  f1-score
Class 0       0.846   0.816     0.828
Class 1       0.768   0.820     0.788
macro         0.808   0.814     0.806
weighted      0.824   0.816     0.816
------------------ K Nearest neighbors ----------------
          precision  recall  f1-score
Class 0       0.806   0.778     0.786
Class 1       0.716   0.764     0.732
macro         0.762   0.770     0.760
weighted      0.780   0.768     0.768
------------------ autoML ------------

,DT,SVM,LR,KNN,ML
0,0.87,0.90,0.82,0.82,0.92
1,0.79,0.87,0.90,0.82,0.92
2,0.62,0.69,0.72,0.62,0.72
3,0.79,0.79,0.82,0.74,0.87
4,0.76,0.79,0.82,0.84,0.87
mean,0.766,0.808,0.816,0.768,0.86


In [19]:
%%time
train_cv,clf_report=kfoldcv(model=densenet121(),epochs=epoch,batchsize=batchsize,data=data,lr=0.001,skip_tuning=False,aug=0)
for clf in ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors','autoML']:
    print('------------------',clf,'----------------')
    print(clf_report[0][clf][0])
    
print('--------------------------Accuracy Table-----------------------------------')
clf_report[1]

KeyError: "['data_482', 'data_472', 'data_209', 'data_475', 'data_265', 'data_266', 'data_363', 'data_145', 'data_241', 'data_153', 'data_424', 'data_374', 'data_138', 'data_341', 'data_294', 'data_489', 'data_130', 'data_408', 'data_347', 'data_502', 'data_348', 'data_336', 'data_466', 'data_443', 'data_368', 'data_222', 'data_325', 'data_193', 'data_172', 'data_349', 'data_367', 'data_450', 'data_384', 'data_200', 'data_441', 'data_205', 'data_510', 'data_383', 'data_395', 'data_399', 'data_407', 'data_263', 'data_324', 'data_191', 'data_310', 'data_229', 'data_212', 'data_220', 'data_358', 'data_333', 'data_410', 'data_366', 'data_259', 'data_144', 'data_249', 'data_394', 'data_245', 'data_361', 'data_287', 'data_160', 'data_255', 'data_150', 'data_405', 'data_462', 'data_498', 'data_478', 'data_334', 'data_302', 'data_402', 'data_194', 'data_375', 'data_401', 'data_149', 'data_400', 'data_235', 'data_185', 'data_511', 'data_493', 'data_308', 'data_479', 'data_289', 'data_387', 'dat